In [10]:
# Importing relevant libraries
import pandas as pd
import numpy as np
import re
from collections import Counter
import nltk
import keras
import spacy
import functools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers import Flatten, Masking
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from random import randint
from numpy import array, argmax, asarray, zeros
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
from sklearn import linear_model
warnings.filterwarnings("ignore")

In [11]:
# Reading 4 Data Frames from part 1
df_color = pd.read_csv("color.csv",index_col = 0)
df_fit = pd.read_csv("fit.csv",index_col = 0)
df_occasion = pd.read_csv("occasion.csv",index_col = 0)
df_style = pd.read_csv("style.csv", index_col = 0)

In [12]:
# transforming T/F to binary (we start getting labels from column 6 onwards that is why we just choose those to convert them to 1 and 0)
df_color.iloc[:,6:]= df_color.iloc[:,6:].astype(int)
df_fit.iloc[:,6:]= df_fit.iloc[:,6:].astype(int)
df_occasion.iloc[:,6:]= df_occasion.iloc[:,6:].astype(int)
df_style.iloc[:,6:]= df_style.iloc[:,6:].astype(int)

In [13]:
df_color.head()

,product_id,brand,product_full_name,description,brand_category,details,blacks,pinks,whites,reds,...,grays,golds,navy,yellows,burgundies,purples,browns,multi,oranges,teal
0,01DSECZPAGJJC1EDC79JRBF4WK,Banana Republic,Mock-Neck Sweater Top,Designed worn high-waisted bottom oh-so-now mo...,Unknown,Designed worn high-waisted bottom oh-so-now mo...,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,01DVA59VHYAPT4PVX32NXW91G5,Tibi,Juan Embossed Mules,Tibis Juan embossed mule made shiny black leat...,women:SHOES:MULES,seen Pre-Fall ‘Number runway Heel measure appr...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01DVA4XY7A0QMMSK3V3SBR52J9,Alexandre Birman,Clarita Bow-Embellished Suede Sandals,Alexandre Birmans Clarita sandal quickly risen...,women:SHOES:SANDALS,Heel height measure approximately 50mm Number ...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01DVBP9AHVQTZXJSBNJ0N2NYJP,Khaite,Leather ankle boots,Heel measure approximately 50mm Number inch Bl...,Shoes Boots Ankle,Fits true size take normal size Italian sizing,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01DVBR93Y7KANZE3C09YCTVXDF,Lauren Manoogian,Alpaca-blend scarf,Brown alpaca-blend Number alpaca Number polyam...,Accessories Scarves Scarves,item measurement are Length 136cm Width 32cm,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
doc_color = df_color.brand + df_color.product_full_name + df_color.description + df_color.brand_category + df_color.details
doc_fit = df_fit.brand + df_fit .product_full_name + df_fit.description + df_fit.brand_category + df_fit.details
doc_occasion = df_occasion.brand + df_occasion.product_full_name + df_occasion.description + df_occasion.brand_category + df_occasion.details
doc_style = df_style.brand + df_style.product_full_name + df_style.description + df_style.brand_category + df_style.details

### 1. Count_Vectorizer with Logistic

In [6]:
# 42 models trained

def logistic_model(doc,df,columns):
    
    vectorizer = CountVectorizer(feature_name)
    X = vectorizer.fit(doc)
    #X = vectorizer.transform(X_test) juse for test
    X = X.toarray()
    X = StandardScaler().fit_transform(X)# same for this (separtely)
    data = pd.DataFrame(X, columns=vectorizer.get_feature_names())
    
    models = []
    
    for col in columns:
        y = df[col].values
        #base_accuracy = y.sum()/len(y)
        #base_accuracy = max(base_accuracy,1-base_accuracy)

        data["TARGET"] = y

        train_df, test_df = train_test_split(data)
        X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
        X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

        y_train = train_df["TARGET"]
        y_test = test_df["TARGET"]

        clf =linear_model.LogisticRegression(C=0.001,random_state=None).fit(X_train, y_train)
           
        #models.append[clf]
        #y_pred = clf.predict(X_test)

        #acc = np.mean(y_pred == y_test)
        models.append(clf)
    return X_test


columns_color = ['blacks','pinks','whites','reds','greens','blues','silvers','neutrals','oranges',
                'beiges','grays','golds','navy','yellows','burgundies','purples','browns','multi','teal']
columns_fit   = ['business casual','classic','modern','boho','glam','romantic','casual','androgynous','edgy','retro','athleisure']
columns_occasion = ['day to night','work','weekend','night out','vacation','coldweather','workout'] 
columns_style = ['semi-fitted','relaxed','straight / regular','fitted / tailored','oversized']

model_color = logistic_model(doc_color,df_color,columns_color)
#model_fit = logistic_model(doc_fit,df_fit,columns_fit)
#model_occasion = logistic_model(doc_occasion,df_occasion,columns_occasion)
#model_style = logistic_model(doc_style,df_style,columns_style)

#model_list = model_color+model_fit+model_occasion+model_style



NameError: name 'feature_name' is not defined

In [7]:
model_color

NameError: name 'model_color' is not defined

In [8]:
ylabel = ['blacks','pinks','whites','reds','greens','blues','silvers','neutrals','oranges',
                'beiges','grays','golds','navy','yellows','burgundies','purples','browns','multi','teal'
'business casual','classic','modern','boho','glam','romantic','casual','androgynous','edgy','retro','athleisure'
'day to night','work','weekend','night out','vacation','coldweather','workout'
'semi-fitted','relaxed','straight / regular','fitted / tailored','oversized']

dftest = pd.DataFrame(ylabel,columns = ["color"])

In [9]:
test_brand = "Forever 21"
test_product_full_name = "Jeans size 34 M,"
test_description = "This is a slim jeans"
test_brand_category = "Denim Jeans"
test_details = "Blue color"

test_docs = test_brand +" " + test_product_full_name + " " + test_description + " " + test_brand_category +  " " + test_details

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
# Remove Punctuations
punctuation = "!@#$%^&*()_+<>?:.,;"  
    
for c in test_docs:
    if c in punctuation:
        test_docs = test_docs.replace(c, "")

    
    
# Remove Stopwords  
stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(test_docs) 
test_docs = [w for w in word_tokens if not w in stop_words] 
test_docs = [] 
for w in word_tokens: 
    if w not in stop_words: 
        test_docs.append(w) 

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(test_docs) 
X = X.toarray()
X = StandardScaler().fit_transform(X)
test_data = pd.DataFrame(X, columns=vectorizer.get_feature_names())

test_data
#y_pred = []
#model_list[0]
#model_list[0].predict(test_data)
    
#test

,21,34,blue,color,denim,forever,jeans,size,slim,this
0,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,3.464102,-0.547723,-0.288675,-0.288675,-0.288675
1,3.464102,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.547723,-0.288675,-0.288675,-0.288675
2,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,1.825742,-0.288675,-0.288675,-0.288675
3,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.547723,3.464102,-0.288675,-0.288675
4,-0.288675,3.464102,-0.288675,-0.288675,-0.288675,-0.288675,-0.547723,-0.288675,-0.288675,-0.288675
5,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.547723,-0.288675,-0.288675,-0.288675
6,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.547723,-0.288675,-0.288675,3.464102
7,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.547723,-0.288675,3.464102,-0.288675
8,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,-0.288675,1.825742,-0.288675,-0.288675,-0.288675
9,-0.288675,-0.288675,-0.288675,-0.288675,3.464102,-0.288675,-0.547723,-0.288675,-0.288675,-0.288675


In [ ]:
nlp = spacy.load("en_core_web_md") 

In [ ]:
def MAX_SEQUENCE_LENGTH(list1):
    max = 0
    for i in list1:
        if max<len(i):
            max=len(i)
    return max

def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt', encoding = 'utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

def make_binary_classification_rnn_model(plot=False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(SimpleRNN(units=64, input_shape=(1, MAX_SEQUENCE_LENGTH)))
    model.add(Dense(16))
    model.add(Dense(2, activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()
    return model

def make_lstm_classification_model(plot = False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(units=32, input_shape=(1, MAX_SEQUENCE_LENGTH)))
    model.add(Dense(2, activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()

    return model


In [ ]:
doc_color

### 2. Glove + LSTM Model 

In [ ]:
# Tokenize Text
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(list(doc_color))

# integer encode the documents
encoded_docs = integer_encode_documents(doc_color, tokenizer)

# padding to create equal length sequences
MAX_SEQUENCE_LENGTH = 1000
padded_docs = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(df_color['blacks']))

# train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2)

VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

# Load in GloVe Vectors
embeddings_index = load_glove_vectors()
embeddings_index

# # create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector
        
# define model
model = make_lstm_classification_model()

# fit the model
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
test_docs = [
    "Employees look like they hate their job. Milkshake was like drinking milk. Food was cold and not warm at all",
    "This Mcdonalds is not only in the business of making crappy food and providing even crappier service watch out for the racket they have in the parking lot . If your not careful reading the sign at the the front of the entrance it is going to cost you $195.00 in parking fees. went in to to ask the management they just blew me off. lucky they are in vegas where they dont count on repeat businesssss.",
    "There are better stores without fruit flies in Griffin, GA.",
    "Slowest drive-thru ever. Better option is to go to the location on arlington"
]

test_docs = list(
    map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), test_docs))

encoded_test_sample = integer_encode_documents(test_docs, tokenizer)

padded_test_docs = keras.preprocessing.sequence.pad_sequences(encoded_test_sample, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

model.predict_classes(padded_test_docs)
prediction = model.predict_classes(padded_test_docs)
encoder.inverse_transform(prediction)

### 3. Word2Vec (Equal Weights)

In [ ]:
# Tokenize Text
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(list(doc_color))

# integer encode the documents
encoded_docs = integer_encode_documents(doc_color, tokenizer)

# padding to create equal length sequences
MAX_SEQUENCE_LENGTH = 1000
padded_docs = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(df_color['blacks']))

# train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2)

VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

# Load in GloVe Vectors
embedding_matrix = []
for i in doc_color:
   embedding_matrix.append(nlp(i).vector)
embedding_matrix = np.asarray(embedding_matrix)
# embeddings_index = load_glove_vectors()
# embeddings_index

#create a weight matrix for words in training docs
# embedding_matrix = zeros((VOCAB_SIZE, 100))
# for word, i in tokenizer.word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: # check that it is an actual word that we have embeddings for
#         embedding_matrix[i] = embedding_vector


# define model
model = make_lstm_classification_model2()

# fit the model
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

In [ ]:
nlp(doc).vector

In [ ]:
# test = "black shoes green belt"
# nlp(test).vector

### 3. GLOVE (Unequal Weights)

In [ ]:
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(doc)
# X = X.toarray()
# X.shape

### 5. Self Trained Corpus

In [ ]:
# doc = list(doc.values)
# doc = [word_tokenize(review) for review in doc]
# model = Word2Vec(doc, min_count=5)
# words = list(model.wv.vocab)
# vectors = []
# for word in words:
#     vectors.append(model[word].tolist())
# data = np.array(vectors)
# data

## Model 1 count vectorizer

### 5. Using count vectorization to find out more words that lemmatization could not remove and assigning them to base form for the purpose of dimensionality reduction

In [ ]:
# converting remaning unchanged words to their base form manually
# doc1 = re.sub(r'wearability|wearable|wearin|wearing','wear',doc1)
# doc1 = re.sub(r'transitioning|transitioned|transitional','transition',doc1)

In [ ]:
list(df_style.columns)

In [ ]:
# # Count vectorization for full data

# # Subset of the broader category



# doc = df_occasion.brand + df_occasion.product_full_name + df_occasion.description + df_occasion.brand_category + df_occasion.details

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(doc) 
# X = X.toarray()
# columns = ['day to night','work','weekend','night out','vacation','coldweather','workout']


# data = pd.DataFrame(X, columns=vectorizer.get_feature_names())

# accuracy = []
# for col in columns:
#     y = df_occasion[col].values
#     base_accuracy = y.sum()/len(y)
#     base_accuracy = max(base_accuracy,1-base_accuracy)
    
#     data["TARGET"] = y

#     train_df, test_df = train_test_split(data)
#     X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
#     X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

#     y_train = train_df["TARGET"]
#     y_test = test_df["TARGET"]

#     lr.fit(X_train, y_train)
#     y_pred = lr.predict(X_test)

#     acc = np.mean(y_pred == y_test)
#     accuracy.append([col,acc,base_accuracy])
# accuracy
# # X= X.toarray()
# # countVector = pd.DataFrame(X, columns=vectorizer.get_feature_names())
# # pd.set_option('display.max_columns', None)
# # countVector.head()

In [ ]:
# df_color.head()

In [ ]:
# doc = df_color.brand + df_color.product_full_name + df_color.description + df_color.brand_category + df_color.details

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(doc) 
# X = X.toarray()
# columns = ['blacks',
#  'pinks',
#  'whites',
#  'reds',
#  'greens',
#  'blues',
#  'silvers',
#  'neutrals',
#  'beiges',
#  'grays',
#  'golds',
#  'navy',
#  'yellows',
#  'burgundies',
#  'purples',
#  'browns',
#  'multi',
#  'oranges',
#  'teal']


# data = pd.DataFrame(X, columns=vectorizer.get_feature_names())
# accuracy = []
# for col in columns:
#     y = df_color[col].values
#     base_accuracy = y.sum()/len(y)
#     base_accuracy = max(base_accuracy,1-base_accuracy)

    
#     data["TARGET"] = y

#     train_df, test_df = train_test_split(data)
#     X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
#     X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

#     y_train = train_df["TARGET"]
#     y_test = test_df["TARGET"]

#     lr.fit(X_train, y_train)
#     y_pred = lr.predict(X_test)

#     acc = np.mean(y_pred == y_test)
#     accuracy.append([col,acc,base_accuracy])
# accuracy

In [ ]:
# doc = df_fit.brand + df_fit.product_full_name + df_fit.description + df_fit.brand_category + df_fit.details

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(doc) 
# X = X.toarray()
# columns = ['business casual',
#  'classic',
#  'modern',
#  'boho',
#  'glam',
#  'romantic',
#  'casual',
#  'androgynous',
#  'edgy',
#  'retro',
#  'athleisure']


# data = pd.DataFrame(X, columns=vectorizer.get_feature_names())
# accuracy = []
# for col in columns:
#     y = df_fit[col].values
#     base_accuracy = y.sum()/len(y)
#     base_accuracy = max(base_accuracy,1-base_accuracy)
    
#     data["TARGET"] = y

#     train_df, test_df = train_test_split(data)
#     X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
#     X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

#     y_train = train_df["TARGET"]
#     y_test = test_df["TARGET"]

#     lr.fit(X_train, y_train)
#     y_pred = lr.predict(X_test)

#     acc = np.mean(y_pred == y_test)
#     accuracy.append([col,acc,base_accuracy])
# accuracy

In [ ]:
# doc = df_style.brand + df_style.product_full_name + df_style.description + df_style.brand_category + df_style.details

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(doc) 
# X = X.toarray()
# columns = [
#  'semi-fitted',
#  'relaxed',
#  'straight / regular',
#  'fitted / tailored',
#  'oversized']


# data = pd.DataFrame(X, columns=vectorizer.get_feature_names())
# accuracy = []
# for col in columns:
#     y = df_style[col].values
#     base_accuracy = y.sum()/len(y)
#     base_accuracy = max(base_accuracy,1-base_accuracy)
#     data["TARGET"] = y

#     train_df, test_df = train_test_split(data)
#     X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
#     X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

#     y_train = train_df["TARGET"]
#     y_test = test_df["TARGET"]
    
#     clf =linear_model.LogisticRegression(C=0.001,random_state=None)
#     clf.fit(X_train, y_train)
#     y_pred = lr.predict(X_test)

#     acc = np.mean(y_pred == y_test)
#     accuracy.append([col,acc,base_accuracy])
# accuracy

In [ ]:
# doc = df_occasion.brand + df_occasion.product_full_name + df_occasion.description + df_occasion.brand_category + df_occasion.details

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(doc) 
# X = X.toarray()
# columns = ['day to night','work','weekend','night out','vacation','coldweather','workout']


# data = pd.DataFrame(X, columns=vectorizer.get_feature_names())

# n=[100,200,300]
# max_depth=[2,4,6,8]
# for i in n:
#     for j in max_depth:
#         accuracy = []
#         for col in columns:
#             y = df_occasion[col].values
#             base_accuracy = y.sum()/len(y)
#             base_accuracy = max(base_accuracy,1-base_accuracy)

#             data["TARGET"] = y

#             train_df, test_df = train_test_split(data)
#             X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
#             X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

#             y_train = train_df["TARGET"]
#             y_test = test_df["TARGET"]

#             rf = RandomForestClassifier(max_depth=j, n_estimators = i,  n_jobs = -1,max_features = 10).fit(X_train, y_train)
#             y_pred = rf.predict(X_test)

#             acc = np.mean(y_pred == y_test)
#             accuracy.append([col,acc,base_accuracy])
#         print(f"max_depth: {j}, estimators: {i}\n{accuracy}\n")



In [ ]:
# # Boosted Trees
# doc = df_occasion.brand + df_occasion.product_full_name + df_occasion.description + df_occasion.brand_category + df_occasion.details

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(doc) 
# X = X.toarray()
# columns = ['day to night','work','weekend','night out','vacation','coldweather','workout']


# data = pd.DataFrame(X, columns=vectorizer.get_feature_names())

# n=[100,200,300]
# max_depth=[3,4]
# for i in n:
#     for j in max_depth:
#             accuracy = []
#             for col in columns:
#                 y = df_occasion[col].values
#                 base_accuracy = y.sum()/len(y)
#                 base_accuracy = max(base_accuracy,1-base_accuracy)

#                 data["TARGET"] = y

#                 train_df, test_df = train_test_split(data)
#                 X_train = train_df.loc[:, ~train_df.columns.isin(['TARGET'])]
#                 X_test = test_df.loc[:, ~test_df.columns.isin(['TARGET'])]

#                 y_train = train_df["TARGET"]
#                 y_test = test_df["TARGET"]

#                 bt=GradientBoostingClassifier(n_estimators=i, learning_rate=0.1,max_depth=j).fit(X_train, y_train)
#                 y_pred = bt.predict(X_test)

#                 acc = np.mean(y_pred == y_test)
#                 accuracy.append([col,acc,base_accuracy])
#             print(f"max_depth: {j}, estimators: {i}\n{accuracy}\n")



## Model 2 Word2Vec

In [ ]:
# df.head()

In [ ]:
# ## TF-IDF Weighted Average Word Embeddings

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(doc1)
# X = X.toarray()
# tf_idf = pd.DataFrame(X, columns=vectorizer.get_feature_names())

# # sum the tf idf scores for each document
# tf_idf["TF_IDF_SUM"] = tf_idf.sum(axis=1)
# tf_idf_scores = list(map( lambda x: x.lower(), tf_idf.columns))
# tf_idf_scores

# TESTING SECTION FOR EVALUATION (FOR PROFESSOR)

In [ ]:
test_brand = "Forever 21"
test_product_full_name = "Jeans size 34 M"
test_description = "This is a slim jeans"
test_brand_category = "Denim Jeans"
test_details = "Blue color"



# test_brand = str(input("Enter Brand: "))
# test_product_full_name = str(input("Product_Full_Name: "))
# test_description = str(input("Product Description: "))
# test_brand_category = str(input("Brand Category: "))
# test_details = str(input("Details: "))

# MAX_SEQUENCE_LENGTH = 4

# test_docs = test_brand +" " + test_product_full_name + " " + test_description + " " + test_brand_category +  " " + test_details
# test_docs = list(map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), test_docs))

# encoded_test_sample = integer_encode_documents(test_docs, tokenizer)
# padded_test_docs = keras.preprocessing.sequence.pad_sequences(encoded_test_sample, maxlen=MAX_SEQUENCE_LENGTH, padding='post')


# VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)



# from keras.layers.recurrent import SimpleRNN
# from keras.layers import Flatten, Masking


# def load_glove_vectors():
#     embeddings_index = {}
     
#     with open('glove.6B.100d.txt',encoding = "utf8") as f:
#         for line in f:
#             values = line.split()
#             word = values[0]
#             coefs = asarray(values[1:], dtype='float32')
#             embeddings_index[word] = coefs
#     print('Loaded %s word vectors.' % len(embeddings_index))
#     return embeddings_index

# labels = 1

# from keras.utils import to_categorical
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# #labels = to_categorical(encoder.fit_transform(labels))

# embeddings_index = load_glove_vectors()

# embedding_matrix = zeros((VOCAB_SIZE, 100))
# for word, i in tokenizer.word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: # check that it is an actual word that we have embeddings for
#         embedding_matrix[i] = embedding_vector
# embedding_matrix

# model = Sequential()
# model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
# model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
# model.add(SimpleRNN(units=64, input_shape=(1, MAX_SEQUENCE_LENGTH)))
# model.add(Dense(32))
# model.add(Dense(9, activation='softmax'))



# prediction = model.predict_classes(padded_test_docs)
# encoder.inverse_transform(prediction)

In [ ]:
# occasion_vectors = []
# for idx, occasion in enumerate(occasions): # iterate through each document
#     tokens = nlp(occasion) # have spacy tokenize the review text
    
#     # initially start a running total of tf-idf scores for a document
#     total_tf_idf_score_per_document = 0
    
#     # start a running total of initially all zeroes (300 is picked since that is the word embedding size used by word2vec)
#     running_total_word_embedding = np.zeros(300) 
#     for token in tokens: # iterate through each token
    
#     # if the token has a pretrained word embedding it also has a tf-idf score
#         if token.has_vector and token.text.lower() in available_tf_idf_scores:
            
#             tf_idf_score = tf_idf_lookup_table.loc[idx, token.text.lower()]
#             #print(f"{token} has tf-idf score of {tf_idf_lookup_table.loc[idx, token.text.lower()]}")
#             running_total_word_embedding += tf_idf_score * token.vector
            
#             total_tf_idf_score_per_document += tf_idf_score
    
#     # divide the total embedding by the total tf-idf score for each document
#     document_embedding = running_total_word_embedding / total_tf_idf_score_per_document
#     occasion_vectors.append(document_embedding)
# occasion_vectors